In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import cv2
from skimage import io
from matplotlib import pyplot as plt
import tensorflow as tf 

## IMAGE RESIZING AND APPLYING CLAHE TECHNIQUE

In [18]:


import cv2
import glob

path = glob.glob('drive/MyDrive/Brain_tumor_detection/brain/yes/*.jpg')
for file in path:
  img2 = cv2.imread(file)
  img = cv2.resize(img2,(256,256))
  lab_img= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
  l, a, b = cv2.split(lab_img)
  clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
  clahe_img = clahe.apply(l)
#plt.hist(clahe_img.flat, bins=100, range=(0,255))

#Combine the CLAHE enhanced L-channel back with A and B channels
  updated_lab_img2 = cv2.merge((clahe_img,a,b))

#Convert LAB image back to color (RGB)
  CLAHE_img = cv2.cvtColor(updated_lab_img2, cv2.COLOR_LAB2BGR)

  cv2.imwrite(file,CLAHE_img)

In [19]:
path = glob.glob('drive/MyDrive/Brain_tumor_detection/brain/no/*.jpg')
for file in path:
  img2 = cv2.imread(file)
  img = cv2.resize(img2,(256,256))
  lab_img= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
  l, a, b = cv2.split(lab_img)
  clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
  clahe_img = clahe.apply(l)
#plt.hist(clahe_img.flat, bins=100, range=(0,255))

#Combine the CLAHE enhanced L-channel back with A and B channels
  updated_lab_img2 = cv2.merge((clahe_img,a,b))

#Convert LAB image back to color (RGB)
  CLAHE_img = cv2.cvtColor(updated_lab_img2, cv2.COLOR_LAB2BGR)

  cv2.imwrite(file,CLAHE_img)
